# 随机映射（Random Projection, RP）

## 符号定义

|符号|含义|
|:--:|:--:|
|$\pmb{x}$|数据点|
|$\pmb{y}$|降维后数据点|
|$S$|数据点集合|
|$N$|数据点总数|
|$f$|映射函数|
|$d$|降维前维度|
|$m$|降维后维度|
|$\epsilon$|常数|
|$Z$|映射矩阵|

## 概念

随机映射尝试了一个非常大胆的降维思路：通过一个随机生成的映射函数直接将高维空间中的数据点嵌入到低维空间，并且保证数据点之间的欧式距离变化存在一个上界和下界（失真可控）。

降维方法往往需要基于一定的假设或者说降维的目标，例如PCA的一种解释是PCA以最大方差的方向对原始数据进行降维，LLE则假设降维前后每一个数据点均可以由其周围的点以相同的权重线性表示。随机映射则保证数据点之间的降维前后欧式距离的变化存在一个上界和下界。

随机映射最为吸引人的地方无疑是其映射函数的随机性。无需计算距离或是特征向量的特点显然能大大加快降维的速度。

随机映射的理论保证是Johnson–Lindenstrauss lemma，Johnson–Lindenstrauss lemma的内容如下：

假设存在一个数据集，$S=\{\pmb{x_i}\}_{i=1}^N, \pmb{x_i}\in\mathcal{R}^d$，d是一个相当高的维数。给定一个常数$\epsilon, 0\leq\epsilon\leq 1$，则存在一个映射函数$f:\mathcal{R}^d\rightarrow\mathcal{R}^m, m\geq\frac{4\ln(N)}{\frac{\epsilon^2}{2} - \frac{\epsilon^3}{3}}$，有如下结论：

$$
\begin{equation}
    (1-\epsilon)||\pmb{x_i}-\pmb{x_j}||^2\leq||f(\pmb{x_i})-f(\pmb{x_j})||^2\leq(1+\epsilon)||\pmb{x_i}-\pmb{x_j}||^2, \forall \pmb{x_i}, \pmb{x_j}\in S
\end{equation}
$$

Johnson–Lindenstrauss lemma表明对于一个高维空间的降维问题，总能找到一个映射函数使得降维后任何两个数据点之间的距离是原空间中这两个数据点欧式距离的$[1-\epsilon, 1+\epsilon]$倍范围内。

在Johnson–Lindenstrauss lemma的基础上，又能证明从高斯分布$\mathcal{N}(0, \frac{1}{\sqrt m})$随机采样生成的映射矩阵大概率正好能满足上述条件。

因此使用从高斯分布$\mathcal{N}(0, \frac{1}{\sqrt m})$随机采样生成的映射矩阵对数据进行降维能够在Johnson–Lindenstrauss lemma的保证下实现较为可靠的降维。

## 算法流程

* 定义数据集、数据维度以及常数$\epsilon$
* 指定降维后维度$m$
* 从高斯分布$\mathcal{N}(0, \frac{1}{\sqrt m})$随机生成映射矩阵$Z$
* 映射得到输出

## 参考资料

https://zhuanlan.zhihu.com/p/158512304

https://scikit-learn.org/stable/modules/random_projection.html

https://scikit-learn.org/stable/auto_examples/miscellaneous/plot_johnson_lindenstrauss_bound.html#sphx-glr-auto-examples-miscellaneous-plot-johnson-lindenstrauss-bound-py

https://web.njit.edu/~usman/courses/cs675_fall19/JL_lemma.pdf

In [1]:
import numpy as np
from sklearn import random_projection

In [2]:
class MyRP(object):

    def __init__(self, eps=0.1, random_state=None):
        self.eps = eps
        self.random_state = random_state
    
    def fit_transform(self, input_data):

        input_data = np.array(input_data)
        n_samples, input_dims = input_data.shape

        min_dims = self.judge_min_dim(n_samples, self.eps)

        if self.random_state is not None:
            np.random.seed(self.random_state)

        # create transfer matrix
        transfer_mat = np.random.normal(loc=0, scale=1/np.sqrt(min_dims), size=(min_dims, input_dims))

        output_mat = np.matmul(input_data, transfer_mat.T)
        return output_mat

    def judge_min_dim(self, n_samples, eps):
        return int(4 * np.log(n_samples)/(pow(eps, 2)/2 - pow(eps, 3)/3))

In [3]:
# --------------------- create data -----------------------
sample_num = 100
input_dims = 10000
eps = 0.5

input_data = np.random.rand(sample_num, input_dims)

# -------------------- sklearn RP --------------------------
sklearn_rp = random_projection.GaussianRandomProjection(eps=0.5, random_state=1024)
sklearn_rp_result = sklearn_rp.fit_transform(input_data)

# --------------------- My RP -----------------------------
my_rp = MyRP(eps=0.5, random_state=1024)
my_rp_result = my_rp.fit_transform(input_data)

# compare result
print(my_rp_result - sklearn_rp_result)
print(np.linalg.norm(sklearn_rp_result-my_rp_result, ord='fro'))

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
0.0
